In [60]:
import pandas as pd
import os

import warnings
warnings.filterwarnings('ignore')

In [49]:
# CO2 emissions per combustion engine car per month
SINGLE_CAR_EMISSIONS = float(4.6/12)

# CA hourly electricity demand in MWh, 2015-2022
ca_hourly_demand = pd.read_csv("data/ca_hourly_demand.csv")

# CA monthly electricity demand in thousand of MWh, 2001-2022
ca_monthly_demand = pd.read_csv("data/ca_monthly_demand.csv")

# CA monthly CO2 emissions in MTCO2, 2014-2022
ca_monthly_emissions = pd.read_csv("data/ca_monthly_emissions.csv")

# CA total yearly vehicle registrations, 2016-2021
ca_yearly_vehicles = pd.read_csv("data/ca_yearly_vehicles.csv")

fuel_types = ['electric','plug-in hybrid electric','fuel cell','diesel','flex fuel','gasoline','gasoline hybrid','natural gas','propane']

In [22]:
def cleanup_null_lines(df: pd.DataFrame, col):
    df = df.dropna()
    df = df[df[col] != ""]
    df = df[df[col] > 0]
    return df

**hourly_emissions_demand.csv**

Compile file (emissions_demand.csv) with CA HOURLY CO2 emissions and hourly electricity demand

In [48]:
DATE_START_IDX = 10
DATE_END_IDX = 18

emissions_demand = ca_hourly_demand
emissions_demand["emissions"] = ""

path_name = "data/ca_daily_emissions"
cwd = os.path.abspath(path_name)
file_list = os.listdir(cwd)

# Sum hourly emissions values from 5-minute interval files
for file in file_list:
    curr_daily_file = pd.read_csv(path_name + "/" + file)
    curr_date = file[DATE_START_IDX:DATE_END_IDX]
    for i in range(24):
        if i > 0: # Does not include 0h values for simplicity
            total_hour_demand = 0
            curr_hour = str(i)
            prev_hour = str(i - 1)
            for i in range(12):
                five_min_increment = str(i * 5)
                if len(five_min_increment) < 2:
                    if i == 0:
                        full_string_hour = curr_hour + ":00"
                    else:
                        full_string_hour = prev_hour + ":0" + five_min_increment
                else:
                    full_string_hour = prev_hour + ":" + five_min_increment
                if full_string_hour not in list(curr_daily_file.columns):
                    full_string_hour = "0" + full_string_hour
                total_hour_demand += curr_daily_file[full_string_hour][0]
            if len(curr_hour) < 2:
                curr_hour = '0' + curr_hour
            full_date = curr_date + "T" + curr_hour + "Z"
            emissions_demand.loc[emissions_demand["date"] == full_date, "emissions"] = total_hour_demand

emissions_demand = cleanup_null_lines(emissions_demand, "emissions")

emissions_demand.to_csv('data/hourly_emissions_demand.csv')
emissions_demand.head()

,date,demand,emissions
0,20221127T23Z,23637,84405
1,20221127T22Z,23599,87295
2,20221127T21Z,23323,88500
3,20221127T20Z,23718,88878
4,20221127T19Z,25038,90162


**yearly_all_data.csv**

Compile file (emissions_vehicles.csv) with CA YEARLY CO2 emissions and number of registered vehicles

In [23]:
emissions_vehicles = ca_yearly_vehicles
emissions_vehicles["emissions"] = ""

months_list = list(ca_monthly_emissions)
months_list.remove('year')

year_list = list(ca_monthly_emissions['year'])

for year in year_list: 
    annual_emissions = ca_monthly_emissions.loc[ca_monthly_emissions["year"] == year][months_list].sum(axis=1)
    emissions_vehicles.loc[emissions_vehicles["year"] == year, "emissions"] = float(annual_emissions)

emissions_vehicles = cleanup_null_lines(emissions_vehicles, "emissions")
emissions_vehicles.to_csv('data/emissions_vehicles.csv')

emissions_vehicles


,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,emissions
0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,53203603.23
1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,49626238.88
2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,51049895.52
3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,53938026.76
4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,52856896.33
5,2016,115569,114602,854,588872,948746,22661954,860599,8521,966,57959115.485
6,2015,82686,83482,79,555652,878165,22026350,776480,8500,895,66241778.651
7,2014,50175,58764,38,524118,817338,21737732,701945,8448,946,68780731.98


Add electricity demand to above file (yearly_all_data.csv)

In [511]:
ev = pd.read_csv("data/emissions_vehicles.csv")

yearly_all_data = ev
yearly_all_data['demand'] = ""

yearly_demand = {}
for index, row in ca_hourly_demand.iterrows():
    curr_year = row['date'][0:4]
    if curr_year in yearly_demand:
        yearly_demand[curr_year] += row['demand']
    else:
        yearly_demand[curr_year] = row['demand']

for year in yearly_demand:
    yearly_all_data.loc[yearly_all_data["year"] == int(year), "demand"] = yearly_demand[year]

yearly_all_data = cleanup_null_lines(yearly_all_data, "demand")
yearly_all_data.to_csv('data/yearly_all_data.csv')
yearly_all_data.head()


,Unnamed: 0,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,emissions,demand
0,0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,53203603.23,269006652
1,1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,49626238.88,268529517
2,2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,51049895.52,264368288
3,3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,53938026.76,275818547
4,4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,52856896.33,284621714


**yearly_vehicles_total.csv**

In [63]:
yearly_vehicles_total = ca_yearly_vehicles
yearly_vehicles_total['total'] = ''

vehicle_list = list(yearly_vehicles_total)
vehicle_list.remove('year')

for index, row in yearly_vehicles_total.iterrows():
    year = row['year']
    annual_vehicles = yearly_vehicles_total.loc[yearly_vehicles_total["year"] == year][vehicle_list].sum(axis=1)
    yearly_vehicles_total.loc[yearly_vehicles_total["year"] == year, 'total']= int(annual_vehicles)

yearly_vehicles_total.to_csv('data/yearly_vehicles_total.csv')
yearly_vehicles_total.head()

,year,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane,total
0,2021,522445,305315,10127,590216,1208970,25998618,1298275,8461,90,29942517
1,2020,369364,259109,7129,700366,1144536,25021380,1155477,7676,897,28665934
2,2019,308447,251522,6933,680622,1131474,25526368,1114944,8462,1015,29029787
3,2018,232239,216328,5525,657857,1122312,25377093,1060166,8910,1063,28681493
4,2017,159705,163057,3271,640281,1108886,25313468,1019085,9212,1074,28418039


**monthly_all_data.csv**

Compile file (monthly_all_data.csv) with CA MONTHLY electricity demand, car registrations, and CO2 emissions

In [43]:
# Sum total electricity generation for each month, converted to megawatthours

CONVERSION_FACTOR = 1000

monthly_all_data = ca_monthly_demand
col_list = ["coal","natural gas","nuclear","hydroelectric","wind"]
monthly_all_data['total'] = ""
monthly_all_data['total'] = monthly_all_data[col_list].sum(axis=1)*CONVERSION_FACTOR

monthly_all_data.head()

,date,coal,date1,natural gas,date2,nuclear,date3,hydroelectric,date4,wind,total
0,202209,24.98724,202209,10736.90153,202209,1611.663,202209,1732.27996,202209,887.64266,14993474.39
1,202208,19.65788,202208,11568.06973,202208,1679.400,202208,1756.68800,202208,1083.22678,16107042.39
2,202207,22.38795,202207,9504.45244,202207,1684.102,202207,1856.69645,202207,1443.68919,14511328.03
3,202206,19.56353,202206,7134.78680,202206,1633.288,202206,1816.49729,202206,1460.87295,12065008.57
4,202205,15.52536,202205,5208.03935,202205,1692.998,202205,1835.48466,202205,1814.01024,10566057.61


In [44]:
monthly_all_data = monthly_all_data.drop(["coal","date1","natural gas","date2","nuclear","date3","hydroelectric","date4","wind"], axis=1)
monthly_all_data = monthly_all_data.rename(columns={"total": "demand"})

months_list = list(ca_monthly_emissions)
months_list.remove('year')

monthly_all_data["emissions"] = ""

for index, row in monthly_all_data.iterrows():
    curr_date = str(row['date'])
    curr_year = curr_date[0:4]
    curr_month = curr_date[4:6]
    if int(curr_year) in ca_monthly_emissions["year"].tolist():
        month_index = int(curr_month) - 1
        month_emissions = ca_monthly_emissions.loc[ca_monthly_emissions['year'] == int(curr_year), months_list[month_index]]
        monthly_all_data.loc[monthly_all_data['date'] == int(curr_date), "emissions"] = float(month_emissions)

monthly_all_data = cleanup_null_lines(monthly_all_data, "emissions")
monthly_all_data.head()


,date,demand,emissions
6,202203,10086091.07,3490823.36
7,202202,9625676.18,3513663.33
8,202201,10918131.70,4339494.42
9,202112,11489598.82,4959996.71
10,202111,10731961.28,4365100.13


In [45]:
monthly_all_data["date_idx"] = ""

dates_list = monthly_all_data['date'].tolist()
dates_list_chronological = dates_list.reverse()
dates_dict = {}

for i in range(len(dates_list)):
    dates_dict[dates_list[i]] = i

for index, row in monthly_all_data.iterrows():
    curr_date = str(row['date'])
    curr_year = curr_date[0:4]
    if int(curr_year) in ca_monthly_emissions["year"].tolist():
        monthly_all_data.loc[monthly_all_data['date'] == int(curr_date), "date_idx"] = dates_dict[int(curr_date)]

monthly_all_data.head()

,date,demand,emissions,date_idx
6,202203,10086091.07,3490823.36,98
7,202202,9625676.18,3513663.33,97
8,202201,10918131.70,4339494.42,96
9,202112,11489598.82,4959996.71,95
10,202111,10731961.28,4365100.13,94


In [46]:
# Estimate monthly car data

MILLION_CONVERSION_FACTOR = 1000000

for type in fuel_types:
    monthly_all_data[type] = ""

year_list = ca_yearly_vehicles['year'].tolist()
for index, row in ca_yearly_vehicles.iterrows():
    curr_year = row['year']
    prev_year = curr_year - 1
    if prev_year in year_list and curr_year in year_list:
        for type in fuel_types:
            curr_year_count = ca_yearly_vehicles.loc[ca_yearly_vehicles['year'] == curr_year, type]
            prev_year_count = ca_yearly_vehicles.loc[ca_yearly_vehicles['year'] == prev_year, type]
            increase = int(curr_year_count) - int(prev_year_count)
            increment = increase / 12
            for i in range(12):
                month = int(i) + 1
                month_string = str(month)
                if len(month_string) < 2:
                    month_string = "0" + month_string
                full_date = str(curr_year) + month_string
                monthly_value = int(prev_year_count) + int(increment) * int(month)
                monthly_all_data.loc[monthly_all_data['date'] == int(full_date), type] = int(float(monthly_value))
    
# for index, row in monthly_all_data.iterrows():
#     gas_car_emissions = row['gasoline'] * SINGLE_CAR_EMISSIONS / MILLION_CONVERSION_FACTOR
#     #emissions_no_gas = row['emissions'] - (row['gasoline'] * SINGLE_CAR_EMISSIONS / MILLION_CONVERSION_FACTOR)
#     monthly_all_data.loc[monthly_all_data['date'] == row['date'], "gas car emissions"] = gas_car_emissions

#monthly_all_data = monthly_all_data.drop(['emissions without gas cars','gas car emissions'], axis=1)
monthly_all_data = cleanup_null_lines(monthly_all_data, "electric")
monthly_all_data.to_csv('data/monthly_all_data.csv')
monthly_all_data.head()


,date,demand,emissions,date_idx,electric,plug-in hybrid electric,fuel cell,diesel,flex fuel,gasoline,gasoline hybrid,natural gas,propane
9,202112,11489598.82,4959996.71,95,522436,305309,10117,590218,1208964,25998612,1298265,8456,93
10,202111,10731961.28,4365100.13,94,509680,301459,9868,599397,1203595,25917176,1286366,8391,160
11,202110,12062383.69,4577134.35,93,496924,297609,9619,608576,1198226,25835740,1274467,8326,227
12,202109,13537589.39,5070292.57,92,484168,293759,9370,617755,1192857,25754304,1262568,8261,294
13,202108,15754584.69,5607233.68,91,471412,289909,9121,626934,1187488,25672868,1250669,8196,361
